In [47]:
%pwd 

'/workspaces/OpenDVCW'

In [48]:
# %cd /home/ubu-admin/Developer/tensorflow-wavelets
%cd /workspaces/OpenDVCW
import OpenDVCW
import numpy as np
import load
import tensorflow as tf
import matplotlib.pyplot as plt
import DataGen
import Callbacks
import datetime

/workspaces/OpenDVCW


In [49]:
BATCH_SIZE = 1
EPOCHS = 500
STEPS_PER_EPOCH = 100
Height = 240
Width = 240
Channel = 3
lmbda = 4096
lr_init = 1e-8
early_stop = 40
I_QP=27

args = OpenDVCW.Arguments()
last = 12
checkponts_last_path = "checkpoints_wavelets_L_{}_{}_{}x{}/".format(lmbda, last, Width, Height)
checkponts_new_path = "checkpoints_wavelets_L_{}_{}_{}x{}/".format(lmbda, last+1, Width, Height)
save_name = "model_save_wavelets_L_{}_{}_{}x{}".format(lmbda, last+1, Width, Height)


In [50]:
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [51]:
rm -rf ./logs/

In [52]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [53]:
model = OpenDVCW.OpenDVC(width=Width, height=Height, batch_size=BATCH_SIZE, num_filters=128, lmbda=lmbda)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_init),)
print("* [Model compiled]...")

print("* [Loading dataset]...")
data = DataGen.DataVimeo90kGenerator("folder_cloud_test.npy", 
                                    BATCH_SIZE,
                                    (Height,Width,Channel),
                                    Channel,
                                    True, 
                                    I_QP,
                                    True)

print("Loading weights")
model.load_weights(checkponts_last_path)


* [Model compiled]...
* [Loading dataset]...
Loading weights


In [54]:
# model.layers[0].trainable = False
# model.layers[1].trainable = False
# model.layers[2].trainable = True
# model.layers[3].trainable = True
# model.layers[4].trainable = False
# model.layers[5].trainable = True

In [55]:
for layer in model.layers:
    print(layer.name, layer.trainable)

mv_analysis True
mv_synthesis True
res_analysis True
res_synthesis True
wavelets_optical_flow_4 True
motion_compensation_4 True


In [56]:
# tf.config.run_functions_eagerly(True)
hist = model.fit(x=data, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS, verbose=1, batch_size=BATCH_SIZE,
                callbacks=[
                    Callbacks.MemoryCallback(),
                    Callbacks.LearningRateReducer(),
                    tf.keras.callbacks.ModelCheckpoint(filepath=checkponts_new_path, save_weights_only=True, save_freq='epoch', monitor="loss", mode='min',  save_best_only=True, verbose=1), 
                    tf.keras.callbacks.TerminateOnNaN(),
                    tf.keras.callbacks.EarlyStopping(monitor='loss', patience=early_stop),
                    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, update_freq="epoch"),            
                    ],
				)

Epoch 1/500
  6/100 [>.............................] - ETA: 19s - loss: 1.6893 - bpp: 0.7375 - mse: 2.3238e-04WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1969s vs `on_train_batch_end` time: 0.2260s). Check your callbacks.


100/100 [==============================] - ETA: 0s - loss: 1.4255 - bpp: 0.7079 - mse: 1.7519e-04[MemoryCallback]:  10094976

Epoch 1: loss improved from inf to 1.42546, saving model to checkpoints_wavelets_L_4096_13_240x240/
100/100 [==============================] - 63s 184ms/step - loss: 1.4255 - bpp: 0.7079 - mse: 1.7519e-04
Epoch 2/500
100/100 [==============================] - ETA: 0s - loss: 1.3263 - bpp: 0.6681 - mse: 1.6069e-04[MemoryCallback]:  10170508

Epoch 2: loss improved from 1.42546 to 1.32629, saving model to checkpoints_wavelets_L_4096_13_240x240/
100/100 [==============================] - 19s 194ms/step - loss: 1.3263 - bpp: 0.6681 - mse: 1.6069e-04
Epoch 3/500
100/100 [==============================] - ETA: 0s - loss: 1.3150 - bpp: 0.6672 - mse: 1.5816e-04[MemoryCallback]:  10223164

Epoch 3: loss improved from 1.32629 to 1.31501, saving model to checkpoints_wavelets_L_4096_13_240x240/
100/100 [==============================] - 18s 183ms/step - loss: 1.3150 - bpp: 

In [57]:
path = load.load_random_path("folder_cloud_test.npy")
i=0
out_bin = "Test_com/test{}.bin".format(i)
out_decom = "Test_com/testdcom{}.png".format(i)
p_on_test = "Test_com/test_p_frame{}.png".format(i)
i_on_test = "Test_com/test_i_frame{}.png".format(i)

i_frame = path + 'im1' + '.png'
p_frame = path + 'im2' + '.png'
print(i_frame)

OpenDVCW.write_png(p_on_test, OpenDVCW.read_png_crop(p_frame, 240, 240))
OpenDVCW.write_png(i_on_test, OpenDVCW.read_png_crop(i_frame, 240, 240))

OpenDVCW.compress(model, i_frame, p_frame, out_bin, 240, 240)
OpenDVCW.decompress(model, i_frame, out_bin, out_decom, 240, 240)

/mnt/WindowsDev/DataSets/vimeo_septuplet/sequences/00088/0172/im1.png
compress
in the compress
decompress
in decompress


In [58]:
model.save(save_name, save_format="tf")

in the compress
in decompress


INFO:tensorflow:Assets written to: model_save_wavelets_L_4096_13_240x240/assets


INFO:tensorflow:Assets written to: model_save_wavelets_L_4096_13_240x240/assets


In [59]:
import cv2
import os
from math import log10, sqrt

def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr


path = load.load_path_n("folder_cloud_test.npy", 0)
p_frame_out_bin = "Test_com/dvcw/p_frame_dvcw.bin"
out_decom = "Test_com/dvcw/frame1.png"
i_on_test = "Test_com/frame0.png"
p_on_test = "Test_com/frame1.png"

i_frame = path + 'im1' + '.png'
p_frame = path + 'im2' + '.png'
print(i_frame)
print(p_frame)

# write inputs to disk
OpenDVCW.write_png(p_on_test, OpenDVCW.read_png_crop(p_frame, 240, 240))
OpenDVCW.write_png(i_on_test, OpenDVCW.read_png_crop(i_frame, 240, 240))


OpenDVCW.compress(model, i_frame, p_frame, p_frame_out_bin, 240, 240)
OpenDVCW.decompress(model, i_frame, p_frame_out_bin, out_decom, 240, 240)


original = cv2.imread(p_on_test)
compressed = cv2.imread(out_decom)
bin_size = os.path.getsize(p_frame_out_bin)
value = PSNR(original, compressed)
print("bin size: ", bin_size , "psnr: ", value)


/mnt/WindowsDev/DataSets/vimeo_septuplet/sequences/00049/0289/im1.png
/mnt/WindowsDev/DataSets/vimeo_septuplet/sequences/00049/0289/im2.png
compress
decompress
bin size:  2398 psnr:  41.02665227844598
